In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o-mini', api_key=api_key)

In [2]:
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

In [4]:
writer = AssistantAgent(
    name="Writer",
    description="A writer agent that generates text based on user input.",
    model_client=model_client,
    system_message="You are a creative writer. Please write a story based on the user's input.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    description="A reviewer agent that provides feedback on the text generated by the writer.",
    model_client=model_client,
    system_message="You are a reviewer. Please provide feedback on the text generated by the writer.",
)

In [5]:
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

graph = builder.build()

In [6]:
graph

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Reviewer', condition=None, condition_function=None, activation_group='Reviewer', activation_condition='all')], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

In [7]:
team = GraphFlow([writer,reviewer], graph)

In [8]:
stream = team.run_stream(task ="Write a good poem about India in less than 30 words.")

async for event in stream:
    print(event)

id='9c546d4a-813f-4c81-93d6-bed762163895' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 8, 14, 14, 10, 80633, tzinfo=datetime.timezone.utc) content='Write a good poem about India in less than 30 words.' type='TextMessage'
id='b52d6f87-a6ac-4851-98d0-2916f1cd5d72' source='Writer' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=35) metadata={} created_at=datetime.datetime(2025, 8, 8, 14, 14, 13, 710657, tzinfo=datetime.timezone.utc) content="Land of colors and ancient grace,  \nLush green fields and desert's face.  \nWhispers of history in every breeze,  \nIndia's heart will never cease.  " type='TextMessage'
id='2c0680a8-3f52-47b4-84e2-7fd1ef0921ff' source='Reviewer' models_usage=RequestUsage(prompt_tokens=82, completion_tokens=152) metadata={} created_at=datetime.datetime(2025, 8, 8, 14, 14, 17, 891821, tzinfo=datetime.timezone.utc) content='Your poem beautifully captures the essence of India, highlighting its diversity and rich history

In [10]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def main():
    # Initialize agents with OpenAI model clients.
    model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
    agent_a = AssistantAgent("A", model_client=model_client, system_message="You are a helpful assistant.")
    agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to Chinese.")
    agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Japanese.")

    # Create a directed graph with fan-out flow A -> (B, C).
    builder = DiGraphBuilder()
    builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
    builder.add_edge(agent_a, agent_b).add_edge(agent_a, agent_c)
    graph = builder.build()

    # Create a GraphFlow team with the directed graph.
    team = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(5),
    )

    # Run the team and print the events.
    async for event in team.run_stream(task="Write a short story about a cat."):
        print(event)


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [11]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient



    # Initialize agents with OpenAI model clients.
model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
agent_a = AssistantAgent("A", model_client=model_client, system_message="You are a helpful assistant.")
agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to Chinese.")
agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Japanese.")
agent_d = AssistantAgent("D", model_client=model_client, system_message="Summarize.")


# Create a directed graph with fan-out flow A -> (B, C).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c).add_node(agent_d)
builder.add_edge(agent_a, agent_b).add_edge(agent_a, agent_c)
builder.add_edge(agent_b, agent_d).add_edge(agent_c, agent_d)
graph = builder.build()

# Create a GraphFlow team with the directed graph.
team = GraphFlow(
    participants=[agent_a, agent_b, agent_c, agent_d],
    graph=graph,
    termination_condition=MaxMessageTermination(5),
)

# Run the team and print the events.
async for event in team.run_stream(task="Write a short story about a cat."):
    print(event)

id='0893c804-ab05-46a5-8dec-9ed8e2bb3547' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 8, 14, 15, 59, 131769, tzinfo=datetime.timezone.utc) content='Write a short story about a cat.' type='TextMessage'
id='c11e00a8-9638-4b74-a3d6-e38669af210f' source='A' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=158) metadata={} created_at=datetime.datetime(2025, 8, 8, 14, 16, 6, 18488, tzinfo=datetime.timezone.utc) content="Once upon a time in a cozy village, there was a curious gray cat named Whiskers. She loved to explore every nook and cranny of her neighborhood, from the sunlit rooftops to the quiet gardens. One sunny morning, Whiskers discovered a tiny bluebird trapped in a thicket. Without hesitation, she gently pried the branches apart and set the bird free.\n\nThe bluebird chirped happily and flitted away, and Whiskers felt a warm glow in her heart. From then on, she became known as the kind-hearted guardian of the village. Each day, she

In [12]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def main():
    # Initialize agents with OpenAI model clients.
    model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
    agent_a = AssistantAgent(
        "A",
        model_client=model_client,
        system_message="Detect if the input is in Chinese. If it is, say 'yes', else say 'no', and nothing else.",
    )
    agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to English.")
    agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Chinese.")

    # Create a directed graph with conditional branching flow A -> B ("yes"), A -> C (otherwise).
    builder = DiGraphBuilder()
    builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
    # Create conditions as callables that check the message content.
    builder.add_edge(agent_a, agent_b, condition=lambda msg: "yes" in msg.to_model_text())
    builder.add_edge(agent_a, agent_c, condition=lambda msg: "yes" not in msg.to_model_text())
    graph = builder.build()

    # Create a GraphFlow team with the directed graph.
    team = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(5),
    )

    # Run the team and print the events.
    async for event in team.run_stream(task="AutoGen is a framework for building AI agents."):
        print(event)


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop